<a href="https://colab.research.google.com/github/alienverarslan/Movie_Recommender_Systems/blob/main/Item_to_Item_CollaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

In [5]:
data = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [6]:
data.describe()

,userID,movieID,rating
count,990209.000000,990209.00000,990209.000000
mean,3016.430061,1865.56350,3.581463
std,1728.237347,1096.00108,1.117173
min,1.000000,1.00000,1.000000
25%,1565.000000,1030.00000,3.000000
50%,2970.000000,1835.00000,4.000000
75%,4535.000000,2770.00000,4.000000
max,6040.000000,3952.00000,5.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990209 entries, 0 to 990208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userID   990209 non-null  int64
 1   movieID  990209 non-null  int64
 2   rating   990209 non-null  int64
dtypes: int64(3)
memory usage: 22.7 MB


In [52]:
#Train test split
test = data.sample(2000, random_state=42)
train = data.drop(test.index)
U_train = train.pivot("movieID", "userID", "rating")

In [54]:
print("Train Utility Matrix:", U_train.shape)
print("All Data Utility Matrix:", data.pivot("movieID", "userID", "rating").shape)

Train Utility Matrix: (3705, 6040)
All Data Utility Matrix: (3705, 6040)


In [55]:
#Train Set Utility Matrix

U_train.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
#Define the Algorithm
def CollaborativeFiltering_ItoI(train_df, test_df, k=20, met='minkowski', avg='uniform'):
  myPreds = []
  realLabels = []
  for idx in test_df.index:
    movieId = test_df["movieID"].loc[idx]
    userId = test_df["userID"].loc[idx]

    filtered_train = train_df.dropna(subset=[userId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)    

    if k < len(filtered_train):
      nn = nei.NearestNeighbors(n_neighbors=k, metric=met, n_jobs=-1)
      nn.fit(filtered_train)   

      test_row = train_df.loc[movieId:movieId, :].copy()
      test_row[userId] = np.nan
      test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
      test_row.fillna(0, inplace=True)

      distances, indices = nn.kneighbors(test_row, return_distance=True)

      movieIds = filtered_train.iloc[indices[0], :].index
      
      if avg == 'weighted':
        avgs = np.average(filtered_train.loc[movieIds, userId].values, weights=(1 - distances[0]+0.000001))
      elif avg == 'uniform':
        avgs = filtered_train.loc[movieIds, userId].values.mean() + 0.000001
      
      pred = np.nanmean(train_df.loc[movieId,:].values) + 0.000001 + avgs

    elif len(filtered_train) >= 10:
      nn = nei.NearestNeighbors(n_neighbors=len(filtered_train), metric=met, n_jobs=-1)
      nn.fit(filtered_train) 

      test_row = train_df.loc[movieId:movieId, :].copy()
      test_row[userId] = np.nan
      test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
      test_row.fillna(0, inplace=True)

      distances, indices = nn.kneighbors(test_row, return_distance=True)

      movieIds = filtered_train.iloc[indices[0], :].index
      
      if avg == 'weighted':
        avgs = np.average(filtered_train.loc[movieIds, userId].values, weights=(1 - distances[0]+0.000001))
      elif avg == 'uniform':
        avgs = filtered_train.loc[movieIds, userId].values.mean() + 0.000001

      pred = np.nanmean(train_df.loc[movieId,:].values) + 0.000001 + avgs

    else:
      pred = np.nanmean(train_df.loc[movieId,:].values) + 0.000001


    if pred > 5:
      pred = 5
    if pred < 1:
      pred = 1
  
    myPreds.append(pred)

    real_label = test_df["rating"].loc[idx]
    realLabels.append(real_label)
  
  return [realLabels, myPreds] 
  

In [125]:
#Finding the optimal parameters with the first 500 ratings of the test data
metrics = ['cosine', "minkowski"]
averages = ['uniform', 'weighted']
k_size = [10, 20, 30, 40, 50]
params = []
for m in metrics:
  for avg in averages:
      for k in k_size:
        params.append([m, avg, k])

print("params                    rmse")
results3 = []
for i in range(len(params)):
  labels, preds = CollaborativeFiltering_ItoI(train_df=U_train, test_df=test[:500], met=params[i][0], avg=params[i][1], k=params[i][2])
  results3.append(met.mean_squared_error(labels, preds, squared=False))  
  print(params[i], results3[i])

params                    rmse
['cosine', 'uniform', 10] 0.8531444354244153
['cosine', 'uniform', 20] 0.8546740444119983
['cosine', 'uniform', 30] 0.866989819640677
['cosine', 'uniform', 40] 0.8698350272539898
['cosine', 'uniform', 50] 0.876484000368574
['cosine', 'weighted', 10] 0.8459143198556404
['cosine', 'weighted', 20] 0.8444167323378577
['cosine', 'weighted', 30] 0.8535958932182656
['cosine', 'weighted', 40] 0.8541143396342725
['cosine', 'weighted', 50] 0.8603393800005225
['minkowski', 'uniform', 10] 0.987316078090713
['minkowski', 'uniform', 20] 0.9641877887920394
['minkowski', 'uniform', 30] 0.9493910808990973
['minkowski', 'uniform', 40] 0.9486492911533291
['minkowski', 'uniform', 50] 0.9477347200188923
['minkowski', 'weighted', 10] 0.9884086746640901
['minkowski', 'weighted', 20] 0.9655215885301495
['minkowski', 'weighted', 30] 0.9501056088593748
['minkowski', 'weighted', 40] 0.9492187781672635
['minkowski', 'weighted', 50] 0.947727362979478


In [127]:
#Best Parameters
best_params = params[np.argmin(results3)]
best_score = results3[np.argmin(results3)]
best_params

['cosine', 'weighted', 20]

In [131]:
#Fine Tune the optimal parameters 
metrics = ['cosine']
averages = ['weighted']
k_size = [15, 16, 17, 18, 19]
params = []
for m in metrics:
  for avg in averages:
      for k in k_size:
        params.append([m, avg, k])

print("params                    rmse")
results2 = []
for i in range(len(params)):
  labels, preds = CollaborativeFiltering_ItoI(train_df=U_train, test_df=test[:500], met=params[i][0], avg=params[i][1], k=params[i][2])
  results2.append(met.mean_squared_error(labels, preds, squared=False))  
  print(params[i], results2[i])

params                    rmse
['cosine', 'weighted', 15] 0.8441731034963756
['cosine', 'weighted', 16] 0.8428948595415008
['cosine', 'weighted', 17] 0.8426871060273848
['cosine', 'weighted', 18] 0.8417022595320689
['cosine', 'weighted', 19] 0.8429451472801658


In [132]:
#Fine tuned parameters
best_params = params[np.argmin(results2)]
best_score = results2[np.argmin(results2)]
best_params

['cosine', 'weighted', 18]

In [134]:
#Predict the test data
labels, preds = CollaborativeFiltering_ItoI(train_df=U_train, test_df=test, met=best_params[0], avg=best_params[1], k=best_params[2])

test_rmse = met.mean_squared_error(labels, preds, squared=False)
print("Test RMSE:",test_rmse)  


Test RMSE: 0.8401265813547066
